# # Sem Tech Project Gruppe 1

In [1]:
# pip install sparqlwrapper
# pip install pandas
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?company
WHERE
{
  ?company wdt:P31 wd:Q4830453.
}"""


### Die aktuelle Query, welche zu jeder Company auch noch die URLs liefert aus dem Kästchen 
### mit "besteht aus", also die Subcompanies
query2="""SELECT ?company ?besteht_aus WHERE {
  ?company wdt:P31 wd:Q4830453.
  OPTIONAL { ?company wdt:P1830 ?besteht_aus. }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query2)

# Länge der Liste
print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    # lieber auskommentiert lassen, da sonst extrem lange Liste die beim Scrollen zum Absturz des Tabs führt !!
    #print(result)
    pass

206004


In [3]:
class CompanyObject:
    def __init__(self, englishName, list_Languages, list_Lables, 
                 compType, compSize, compTurnover, ceo_name, cio_name, isSubsidiaryOf,  compID=0):
        self.compID=compID
        self.englishName=englishName
        self.list_Languages=list_Languages
        self.list_Lables=list_Lables
        self.compType=compType
        self.compSize=compSize
        self.compTurnover=compTurnover
        self.ceo_name=ceo_name
        self.cio_name=cio_name
        self.isSubsidiaryOf=isSubsidiaryOf

In [4]:
class LanguageObject:
    def __init__(self, region, alphabet):
        self.region=region
        self.alphabet=alphabet

In [5]:
### Example Instantiates

# Example Instance of Mother Company
NokiaMother = CompanyObject('Nokia',['Englisch','Japan'],['Nokia','JapanNokia'],'AG','More than 1000',
                     'More than 1 Billion','Nok_CEO','Nok_CIO', 'null')

# How to Access it
print("Name: ",NokiaMother.englishName,"\n")

# Example Instance of Daughter Company
NokiaDaughter = CompanyObject('Nokia_Daughter',['Englisch','Japan'],['NokiaDaugh','JapanNokiaDaugh'],'AG','More than 10',
                               'More than 1 Million','Nok_CEODaugh','Nok_CIODaugh', NokiaMother)

# How to Access it # Access to MotherCompany( if available)
print("Name: ",NokiaDaughter.englishName," is a subsidiary of: ",NokiaDaughter.isSubsidiaryOf.englishName)

Name:  Nokia 

Name:  Nokia_Daughter  is a subsidiary of:  Nokia


In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

ListOfAllCompanies=[]


### Here start the process of analyzing currently only ONE element ###

response = requests.get(
	url="https://www.wikidata.org/wiki/Q20716",
)
soup = BeautifulSoup(response.content, 'html.parser')

languages=[]
lables=[]

table = soup.find_all("tr", class_=re.compile("wikibase-entitytermsforlanguageview"))

englishName=table[0].parent.find("div").find("span").contents[0]


for elem in table:
    parentElement=elem.parent
    # Add Language Name to the Array
    languages.append(parentElement.find("th").contents[0])
    # Add Label to the other Array
    lables.append(parentElement.find("div").find("span").contents[0])
    print("I found Language: ",parentElement.find("th").contents[0])
    
# We have all information gathered to this company and add it to the "library"
ListOfAllCompanies.append(CompanyObject(englishName,languages,lables))



I found Language:  English


In [4]:
### Second Try with the JSON File

import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

response = requests.get(
	#url="https://www.wikidata.org/wiki/Special:EntityData/Q1418.json"
    url=url)

text = response.text

data = json.loads(text)

df = pd.DataFrame(data['entities']['Q1418']['labels'])
df

#print(data['entities']['Q1418']['labels'])

,en,fr,it,de,fi,ar,az,be,bg,bn,...,sw,wuu,sat,simple,diq,se,smn,sms,vec,ps
language,en,fr,it,de,fi,ar,az,be,bg,bn,...,sw,wuu,sat,simple,diq,se,smn,sms,vec,ps
value,Nokia,Nokia,Nokia,Nokia,Nokia,نوكيا,Nokia,Nokia,Нокия,নকিয়া,...,Nokia,诺基亚,ᱱᱳᱠᱤᱭᱟ,Nokia,Nokia,Nokia,Nokia,Nokia,Nokia,نوکیا


In [5]:
# Print the whole List:
for com in ListOfAllCompanies:
    print(com.englishName,", ",com.list_Languages,", ",com.list_Lables)

Samsung ,  ['English'] ,  ['Samsung']


In [6]:
import time

def createList(n):
    lst = []
    for i in range(n+1):
        lst.append(i)
    return(lst)

#print(createList(10))
arr=createList(10)

for i in arr:
    print(i)
    time.sleep(0.5)

0
1
2
3
4
5
6
7
8
9
10


In [22]:
### Dont Change STH Here ;)

import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

results = get_results(endpoint_url, query2)

def crawl(compID):
    response = requests.get(
        url=("https://www.wikidata.org/wiki/Special:EntityData/"+compID+".json"))
    data = json.loads(response.text)
    df = pd.DataFrame(data['entities'][compID]['labels'])
    
    print(df)

    languages=df.iloc[0]
    labels=df.iloc[1]
    print(a)
   # compObject = CompanyObject('Test',
   #                           languages, labels,
   #                            )


NokiaDaughter = CompanyObject('Nokia_Daughter',['Englisch','Japan'],['NokiaDaugh','JapanNokiaDaugh'],'AG','More than 10',
                               'More than 1 Million','Nok_CEODaugh','Nok_CIODaugh', NokiaMother)

crawl('Q1418')
# Länge der Liste
#print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    pass

             en     fr     it     de     fi     ar     az     be     bg  \
language     en     fr     it     de     fi     ar     az     be     bg   
value     Nokia  Nokia  Nokia  Nokia  Nokia  نوكيا  Nokia  Nokia  Нокия   

              bn  ...     sw  wuu     sat  simple    diq     se    smn    sms  \
language      bn  ...     sw  wuu     sat  simple    diq     se    smn    sms   
value     নকিয়া  ...  Nokia  诺基亚  ᱱᱳᱠᱤᱭᱟ   Nokia  Nokia  Nokia  Nokia  Nokia   

            vec     ps  
language    vec     ps  
value     Nokia  نوکیا  

[2 rows x 141 columns]
(1, 2)


In [83]:
from owlready2 import *

onto_path.append("/Users/leonhardziegler/Library/Mobile Documents/com~apple~CloudDocs/UniBw/Master/SematischeTech/Übungen/git_new")
#onto.save(file = "filename or fileobj", format = "rdfxml")
onto = get_ontology("file://names_onto.owl").load()
#default_world.set_backend(filename = "names_onto.owl")


In [68]:
###test 

#company_instance = onto.owl:Company("Q1418"))
"""with onto:
    class Language(Thing):
        namespace = onto
        pass

with onto:
    class Dialect(Language):
        namespace = onto
        pass
"""

language_instance = Language("French")

dialect_instance = Language("Bavarian")

print(list(onto.individuals()))

print(language_instance.name)




None
[webprotege.stanford.edu.R1tTpXLvo8OlfGKD1B9sD8, webprotege.stanford.edu.R8ov2vaR2o6xwusUMLP6Ttc, webprotege.stanford.edu.R9Mm4K8OsE35OMG2l8lzmMB, names_onto.English, names_onto.French, names_onto.Bavarian]
French


In [84]:
onto.save()
